In [1]:
import argparse
import numpy as np
import random
import argparse
import os
import time
import math
import sys
import json
import csv
import pandas as pd

import torch
from torch.autograd import Variable

from models import load_models_decode, generate
from utils import to_gpu, Corpus, batchify, train_ngram_lm, get_ppl

In [2]:
# load_path='output/out1102'
# load_path='output/out_twitter1103'
load_path='output/out1106_komoran'

In [3]:
model_args, idx2word, autoencoder, gan_gen, gan_disc, word2idx= load_models_decode(load_path)

Loading models fromoutput/out1106_komoran


In [4]:
vocab2=idx2word
vocab2

{0: '<pad>',
 1: '<sos>',
 2: '<eos>',
 3: '<oov>',
 4: 'ㄴ',
 5: 'ㄴ.',
 6: 'ㄴ가',
 7: 'ㄴ가.',
 8: 'ㄴ가요.',
 9: 'ㄴ걸',
 10: 'ㄴ고',
 11: 'ㄴ다',
 12: 'ㄴ다.',
 13: 'ㄴ다거나',
 14: 'ㄴ다고',
 15: 'ㄴ다기',
 16: 'ㄴ다는',
 17: 'ㄴ다면',
 18: 'ㄴ데',
 19: 'ㄴ데요',
 20: 'ㄴ듯',
 21: 'ㄴ듯.',
 22: 'ㄴ지',
 23: 'ㄴ지.',
 24: 'ㄹ',
 25: 'ㄹ.',
 26: 'ㄹ걸',
 27: 'ㄹ까',
 28: 'ㄹ까.',
 29: 'ㄹ까요',
 30: 'ㄹ까요.',
 31: 'ㄹ려고',
 32: 'ㄹ지',
 33: 'ㄹ지라도',
 34: 'ㅁ',
 35: 'ㅁ.',
 36: 'ㅂ니다',
 37: 'ㅂ니다.',
 38: 'ㅂ사',
 39: '가',
 40: '가.',
 41: '가감.',
 42: '가게',
 43: '가격',
 44: '가격.',
 45: '가공',
 46: '가까이',
 47: '가깝',
 48: '가끔',
 49: '가끔가다',
 50: '가능',
 51: '가능.',
 52: '가득',
 53: '가득.',
 54: '가라앉히',
 55: '가량',
 56: '가루',
 57: '가볍',
 58: '가수',
 59: '가운데',
 60: '가장',
 61: '가제',
 62: '가족',
 63: '가지',
 64: '가치',
 65: '각',
 66: '각인',
 67: '각지',
 68: '간',
 69: '간간히',
 70: '간결',
 71: '간단',
 72: '간단명료',
 73: '간단히',
 74: '간편',
 75: '간혹',
 76: '갈',
 77: '갈리',
 78: '갈비',
 79: '갈색',
 80: '갈수록',
 81: '갈증',
 82: '감',
 83: '감.',
 84: '감각',
 85: '감각.',
 86: '감귤',
 87: '감기',

In [5]:
word2idx

{'<pad>': 0,
 '<sos>': 1,
 '<eos>': 2,
 '<oov>': 3,
 'ㄴ': 4,
 'ㄴ.': 5,
 'ㄴ가': 6,
 'ㄴ가.': 7,
 'ㄴ가요.': 8,
 'ㄴ걸': 9,
 'ㄴ고': 10,
 'ㄴ다': 11,
 'ㄴ다.': 12,
 'ㄴ다거나': 13,
 'ㄴ다고': 14,
 'ㄴ다기': 15,
 'ㄴ다는': 16,
 'ㄴ다면': 17,
 'ㄴ데': 18,
 'ㄴ데요': 19,
 'ㄴ듯': 20,
 'ㄴ듯.': 21,
 'ㄴ지': 22,
 'ㄴ지.': 23,
 'ㄹ': 24,
 'ㄹ.': 25,
 'ㄹ걸': 26,
 'ㄹ까': 27,
 'ㄹ까.': 28,
 'ㄹ까요': 29,
 'ㄹ까요.': 30,
 'ㄹ려고': 31,
 'ㄹ지': 32,
 'ㄹ지라도': 33,
 'ㅁ': 34,
 'ㅁ.': 35,
 'ㅂ니다': 36,
 'ㅂ니다.': 37,
 'ㅂ사': 38,
 '가': 39,
 '가.': 40,
 '가감.': 41,
 '가게': 42,
 '가격': 43,
 '가격.': 44,
 '가공': 45,
 '가까이': 46,
 '가깝': 47,
 '가끔': 48,
 '가끔가다': 49,
 '가능': 50,
 '가능.': 51,
 '가득': 52,
 '가득.': 53,
 '가라앉히': 54,
 '가량': 55,
 '가루': 56,
 '가볍': 57,
 '가수': 58,
 '가운데': 59,
 '가장': 60,
 '가제': 61,
 '가족': 62,
 '가지': 63,
 '가치': 64,
 '각': 65,
 '각인': 66,
 '각지': 67,
 '간': 68,
 '간간히': 69,
 '간결': 70,
 '간단': 71,
 '간단명료': 72,
 '간단히': 73,
 '간편': 74,
 '간혹': 75,
 '갈': 76,
 '갈리': 77,
 '갈비': 78,
 '갈색': 79,
 '갈수록': 80,
 '갈증': 81,
 '감': 82,
 '감.': 83,
 '감각': 84,
 '감각.': 85,
 '감귤': 86,
 '감기': 87,

In [5]:
model_args

{'N': 5,
 'arch_d': '300-300',
 'arch_g': '300-300',
 'batch_size': 5,
 'beta1': 0.9,
 'clip': 1,
 'cuda': True,
 'data_path': 'data_korean',
 'dropout': 0.0,
 'emsize': 300,
 'enc_grad_norm': True,
 'epochs': 49,
 'gan_clamp': 0.01,
 'gan_toenc': -0.01,
 'hidden_init': False,
 'kenlm_path': '../Data/kenlm',
 'log_interval': 200,
 'lowercase': False,
 'lr_ae': 1,
 'lr_gan_d': 1e-05,
 'lr_gan_g': 5e-05,
 'maxlen': 30,
 'min_epochs': 50,
 'nhidden': 300,
 'niters_ae': 1,
 'niters_gan_d': 5,
 'niters_gan_g': 1,
 'niters_gan_schedule': '2-4-6',
 'nlayers': 1,
 'no_earlystopping': False,
 'noise_anneal': 0.995,
 'noise_radius': 0.2,
 'ntokens': 10919,
 'outf': 'out1102',
 'patience': 5,
 'sample': False,
 'seed': 1111,
 'temp': 1,
 'vocab_size': 11000,
 'z_size': 100}

# ---------전처리

In [266]:
import re

In [254]:
file = 'codedata/rawinput.csv'
rawdata = pd.read_csv(file, names = None, encoding='CP949')
rawdoc = rawdata['응답값']

In [265]:
rawdoc1 = [" ".join(re.findall('([가-힣]+)', row))+"." for row in rawdoc]
rawdoc1
with open("train.txt", 'w') as f:
    for s in rawdoc1:
        f.write(str(s) + '\n')

# ------------------

In [7]:
path = 'codedata/clusterdata.csv'
maxlen=50

In [38]:
path = 'codedata/komoran_code_decode/clust_komoran_all.csv'
maxlen=100

In [9]:
data = pd.read_csv(path, names = None)
clusternum=data['ClusterNo']
doc = data['rawdoc']
groupby_cluster = data['rawdoc'].groupby([data['ClusterNo']])
cluster_doc=groupby_cluster.apply(lambda x: x.tolist()).to_dict()

In [10]:
cluster_doc

{0: ['시원해서.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원해서.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원해서.',
  '시원함.',
  '시원하다.',
  '시원해서.',
  '시원해서.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원해서.',
  '시원함.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원해서.',
  '시원해서.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원하다.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원.',
  '시원해서.',
  '시원해서.',
  '시원하다.',
  '시원하다.',
  '시원하다.',
  '시원해서.',
  '시원.',
  '시원하다.',
  '시원해서.',
  '시원해서.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원하다.',
  '시원하다.',
  '시원함.',
  '시원하다.',
  '시원하다.',
  '시원해서.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원해서.',
  '시원하다.',
  '시원함.',
  '시원해요.',
  '시원함.',
  '시원함.',
  '시원함.',
  '시원함.',
  '시원해서.',
  '시원함.',
  '시원해서.',
  '시원해서.',
  '시원하다.',
  '시원함.',
  '시원함.'],
 1: ['맛이 깔끔하다.',
  '맛이 깔끔함.',
  '맛이 깔끔함.',
  '깔끔한 맛.',
  '맛이 깔끔하다.',
  '맛이 깔끔해서.',
  '맛이 깔끔함.',
  '맛이 깔끔함.',
  '맛이 깔끔하다.',
  '깔끔한 맛.',
  '깔끔한 맛.',
  '깔끔한 맛.',
  '깔끔한 

In [11]:
def doc_to_line(cluster_doc):

    dropped = 0
    linecount = 0
    lines = []
    for line in cluster_doc:
        linecount += 1        
#         if self.lowercase:
#             words = line[:-1].lower().strip().split(" ")
#         else:
#             words = line[:-1].strip().split(" ")
        words = line[:-1].strip().split(" ")
        
        if len(words) > maxlen:
            dropped += 1
            continue
        words = ['<sos>'] + words
        words += ['<eos>']

        # vectorize
        vocab = word2idx
        unk_idx = vocab['<oov>']
        indices = [vocab[w] if w in vocab else unk_idx for w in words]

#         print(indices)
        lines.append(indices)
        
    return lines, vocab

In [12]:
def make_code(lines, vocab):
    eval_batch_size = 1
    test_data = batchify(lines, eval_batch_size, shuffle=False)
    ntokens = len(vocab)
    
    
    code = []
    for i, batch in enumerate(test_data):
            source, target, lengths = batch
    #         source = to_gpu(True, Variable(source, volatile=True))
    #         target = to_gpu(True, Variable(target, volatile=True))
    #         mask = target.gt(0)
    #         masked_target = target.masked_select(mask)
    #         # examples x ntokens
    #         output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)

            # output: batch x seq_len x ntokens
            output = autoencoder.encode(Variable(source), lengths, noise=False)
#             print(output)
            code.append(output)
    return code

In [13]:
cluster_title=[]
for c in cluster_doc : 
    lines, vocab = doc_to_line(cluster_doc[c])
    code = make_code(lines, vocab)

    code_cat=torch.cat(code,0)
    code_mean=torch.mean(code_cat,0).view(1,-1) #code 평균

    max_indices = autoencoder.generate_code(code_mean, maxlen)
    max_indices = max_indices.data.cpu().numpy().squeeze()
    

    words = [vocab2[x] for x in max_indices.squeeze()]
#     print(words)
    # truncate sentences to first occurrence of <eos>
    truncated_sent = []
    for w in words:
        if w != '<eos>':
            truncated_sent.append(w)
        else:
            break
    sent = " ".join(truncated_sent)
    cluster_title.append(sent)

RuntimeError: tried to construct a tensor from a nested int sequence, but found an item of type numpy.int32 at index (0, 0)

In [29]:
len(cluster_title)

536

In [44]:
resultdoc={}
for i in range(len(cluster_title)):
    resultdoc[i]=cluster_title[i]

In [14]:
resultdoc

{0: '고소함.',
 1: '맛이 진하다.',
 2: '부드럽다.',
 3: '맛있어요.',
 4: '목넘김이 부드러워서 않아서.',
 5: '톡 쏘는 맛이 깊다.',
 6: '깔끔해요.',
 7: '맛이 진하다.',
 8: '입맛에 가장 딱맞는다.',
 9: '맛이 산뜻하다.',
 10: '진하다.',
 11: '목넘김이 부드러워서 상쾌해서.',
 12: '맛이 깊고 가볍다.',
 13: '그냥 오래된거라.',
 14: '뒷맛이 깔끔하고 예뻐서.',
 15: '시원하고 깔끔하고 예뻐서.',
 16: '맛이 시원하고 깊다.',
 17: '맥주 맛이 깊다.',
 18: '목넘김이 시원하고 좋음',
 19: '깔끔하고 맛있다 무난하다.',
 20: '부드럽고 맛있어서',
 21: '거품이 깊고 맛잇다.',
 22: '맛있어요 가격이 잘넘어감.',
 23: '맛과 향이 깊다.',
 24: '순하고 맛이 좋음',
 25: '달달한 맛이 담백.',
 26: '독특한 맛이 달달.',
 27: '맛이 좋음',
 28: '쓴 맛이 깔끔했다.',
 29: '톡 쏘는 맛이 강하고 맛잇다.',
 30: '깊은 맛이 신선하고 가볍다.',
 31: '맥아의 맛이 좋다 순해서.',
 32: '독특한 맛이 좋다 색이',
 33: '맥주맛과 깊이.',
 34: '국산 맥주 중 맛이 덜해서.',
 35: '내 입맛에 잘 맞는것',
 36: '상표가 쓴 맛에 맛이라서.',
 37: '자주 마시는 거라서.',
 38: '시원한 맛과 디자인.',
 39: '부드럽다 세련되고 예뻐서.',
 40: '목넘김도 부드럽고 고소한 맛이 깊다.',
 41: '진한 맛 맥주맛.',
 42: '대중적이고 쉽게 접할 수 있고 맥주이다.',
 43: '칼로리가 낮고 칼로리가 낮음.',
 44: '제일 맛이 좋음 순해서.',
 45: '맥주특유의 맛이 깔끔해서 브랜드가 잘 마셔진다.',
 46: '청량감이 톡 쏘는 고소함.',
 47: '구수한 맛이 전체적으로 깔끔합니다.',
 48: '호가든 특유의 과일향과',

In [45]:
df = pd.DataFrame([resultdoc,cluster_doc]).T
df['clusterNo'] = df.index

In [16]:
df

,0,1,clusterNo
0,고소함.,"[시원해서., 시원함., 시원하다., 시원함., 시원해서., 시원함., 시원하다.,...",0
1,맛이 진하다.,"[맛이 깔끔하다., 맛이 깔끔함., 맛이 깔끔함., 깔끔한 맛., 맛이 깔끔하다.,...",1
2,부드럽다.,"[부드럽다., 부드러움., 부드러움., 부드러움., 부드러움., 부드럽다., 부드러...",2
3,맛있어요.,"[맛있다., 맛있음., 맛있어서., 맛있어서요., 맛있어요., 맛있다., 맛있어서....",3
4,목넘김이 부드러워서 않아서.,"[부드러운 목넘김이 좋다 맛도 부드러워 좋다., 목넘김이 좋고 부드럽다., 부드럽고...",4
5,톡 쏘는 맛이 깊다.,"[톡 쏘는 맛., 톡 쏘아서., 맛이 좋음 톡쏨., 톡 쏘는 맛이 좋다., 톡 쏘는...",5
6,깔끔해요.,"[깔끔하다., 깔끔해요., 깔끔함., 깔끔함., 깔끔해서., 깔끔해서., 깔끔하므로...",6
7,맛이 진하다.,"[부드러운 맛., 맛이 부드러워서., 부드러운 맛., 맛이 부드럽다., 맛이 부드러...",7
8,입맛에 가장 딱맞는다.,"[맛이 입맛에 맞아서., 내 입맛에 맞아서., 내 입맛에 맞음., 맛이 입맛에 맞아...",8
9,맛이 산뜻하다.,"[맛이 좋다., 맛이 좋아서., 맛이 좋아서., 맛이 좋아서., 맛이 좋아서., 맛...",9


In [46]:
rows = []
_ = df.apply(lambda row: [rows.append([row['clusterNo'],row[0],  nn]) 
                         for nn in row[1]], axis=1)

In [47]:
df_new = pd.DataFrame(rows, columns=['clusterNo','title','raw_docs'])
df_new

,clusterNo,title,raw_docs
0,0,시원 하 ㅁ 맛있 아요.,시원 하 아서.
1,0,시원 하 ㅁ 맛있 아요.,시원 하 ㅁ.
2,0,시원 하 ㅁ 맛있 아요.,시원 하 다.
3,0,시원 하 ㅁ 맛있 아요.,시원 하 ㅁ.
4,0,시원 하 ㅁ 맛있 아요.,시원 하 아서.
5,0,시원 하 ㅁ 맛있 아요.,시원 하 ㅁ.
6,0,시원 하 ㅁ 맛있 아요.,시원 하 다.
7,0,시원 하 ㅁ 맛있 아요.,시원 하 ㅁ.
8,0,시원 하 ㅁ 맛있 아요.,시원 하 아서.
9,0,시원 하 ㅁ 맛있 아요.,시원 하 ㅁ.


In [48]:
df_new.to_csv('k_generate_title_result_raw.csv',sep=',',encoding='CP949')

# ------------------------------------------------

In [8]:
dropped = 0
with open(path, 'r') as f:
    linecount = 0
    lines = []
    for line in f:
        linecount += 1        
#         if self.lowercase:
#             words = line[:-1].lower().strip().split(" ")
#         else:
#             words = line[:-1].strip().split(" ")
        words = line[:-1].strip().split(" ")
        if len(words) > maxlen:
            dropped += 1
            continue
        words = ['<sos>'] + words
        words += ['<eos>']
        # vectorize
        vocab = word2idx
        unk_idx = vocab['<oov>']
        indices = [vocab[w] if w in vocab else unk_idx for w in words]
        lines.append(indices)

In [26]:
dropped = 0
linecount = 0
lines = []
for line in cluster_doc[48]:
    linecount += 1        
#         if self.lowercase:
#             words = line[:-1].lower().strip().split(" ")
#         else:
#             words = line[:-1].strip().split(" ")
    words = line[:-1].strip().split(" ")
    if len(words) > maxlen:
        dropped += 1
        continue
    words = ['<sos>'] + words
    words += ['<eos>']
    # vectorize
    vocab = word2idx
    unk_idx = vocab['<oov>']
    indices = [vocab[w] if w in vocab else unk_idx for w in words]
    lines.append(indices)

In [27]:
eval_batch_size = 1
test_data = batchify(lines, eval_batch_size, shuffle=False)
ntokens = len(word2idx)

In [28]:
code = []
for i, batch in enumerate(test_data):
        source, target, lengths = batch
#         source = to_gpu(True, Variable(source, volatile=True))
#         target = to_gpu(True, Variable(target, volatile=True))
        print(source)
        print(source.shape)
        print(lengths)
#         mask = target.gt(0)
#         masked_target = target.masked_select(mask)
#         # examples x ntokens
#         output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)

        # output: batch x seq_len x ntokens
        output = autoencoder.encode(Variable(source), lengths, noise=False)
        print(output)
        code.append(output)


    1  5020  3048
[torch.LongTensor of size 1x3]

torch.Size([1, 3])
[3]
Variable containing:

Columns 0 to 9 
-0.4109  0.2240 -0.5171  0.1784  0.3945 -0.5976  0.5891  0.3542  0.6336 -0.0092

Columns 10 to 19 
 0.8363 -0.1395 -0.1931 -0.2271 -0.6416 -0.0596  0.6023 -0.7724 -0.4374 -0.0137

Columns 20 to 29 
 0.3264  0.7486 -0.5704 -0.5601  0.0351 -0.5767  0.1553 -0.2205  0.1470  0.2975

Columns 30 to 39 
-0.0740  0.0715 -0.5254 -0.6654 -0.1387 -0.2738  0.1072 -0.8304  0.2127 -0.2477

Columns 40 to 49 
-0.0793  0.1631  0.5389  0.1282 -0.7743 -0.1925 -0.5989  0.1864  0.0158  0.4245

Columns 50 to 59 
 0.6548 -0.1518  0.5294  0.4486  0.7078 -0.1385 -0.1003  0.2794  0.0005  0.2516

Columns 60 to 69 
-0.1091 -0.2002 -0.6102  0.3125 -0.4647 -0.1504  0.4009  0.0655  0.1900 -0.0277

Columns 70 to 79 
 0.2422  0.4097  0.8396 -0.4333 -0.5609 -0.0061 -0.3958 -0.3646  0.2211  0.3314

Columns 80 to 89 
 0.2671 -0.3188  0.2231 -0.6381  0.2386  0.5157 -0.1798  0.5904  0.8183 -0.5584

Columns 90 to 9

In [29]:
code_mean=torch.max(code_cat,0)[0].view(1,-1)

In [30]:
code_cat=torch.cat(code,0)
code_mean=torch.mean(code_cat,0).view(1,-1)

In [31]:
max_indices = autoencoder.generate_code(code_mean, maxlen)
max_indices = max_indices.data.cpu().numpy().squeeze()

In [32]:
words = [vocab2[x] for x in max_indices.squeeze()]
print(words)
# truncate sentences to first occurrence of <eos>
truncated_sent = []
for w in words:
    if w != '<eos>':
        truncated_sent.append(w)
    else:
        break
sent = " ".join(truncated_sent)

['호가든', '특유의', '과일향과', '<eos>', '<eos>', '<eos>', '<eos>', '맛이', '느껴져서.', '<eos>', '<eos>', '<eos>', '향과', '거품도', '느껴져서.', '<eos>', '<eos>', '<eos>', '향과', '맛이', '느껴져서.', '<eos>', '<eos>', '<eos>', '특유의', '청량감.', '<eos>', '<eos>', '<eos>', '톡쏘는']


In [221]:
sent

'카스가 대표적이니까.'